Data formating for PML Model building

In [27]:
##Libraries needed

import numpy as np
import pandas as pd
from keras.layers import Dense
import matplotlib.pyplot as plt
#from sklearn.manifold import TSNE
from keras.optimizers import Adam
from sklearn.decomposition import PCA
from keras.models import Sequential, Model
#from umap import UMAP
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from sklearn.metrics import classification_report,confusion_matrix
import pandas as pd
from collections import Counter
import numpy as np
import scanpy as sc
from matplotlib import pyplot
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.svm import SVC
from sklearn.metrics import *
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import os
%reload_ext jupyternotify

<IPython.core.display.Javascript object>

Gene intersection

In [28]:
##Identify gene intersection between samples

    #Loading the data (1 sample per dataset)

#Rodriguez (FA)
FApath = r'E:/AML_project/DATASETS/Rodriguez/pacientes/genecounts'
df_FA = pd.read_csv(FApath + '/patient1.csv', sep='\t') #read file
df_FA = df_FA.rename({'Unnamed: 0': 'Gene'}, axis=1) #name gene column
df_FA = df_FA.sort_values('Gene')
genes_FA = df_FA['Gene'].to_list()

#van Galen (AML)
    #When using all sequenced genes
AMLpath = r'E:/AML_project/DATASETS/vanGalen/pacientes/genecounts/D0'
df_419 = pd.read_csv(AMLpath +'/GSM3587950_AML419A-D0.dem.txt.gz',sep='\t') #read file
df_419 = df_419.sort_values('Gene') #sort values alphabetically
genes_AML = df_419['Gene'].to_list() #generate list of the genes
    #When using only malignant variable genes
# genes_vGa = pd.read_csv("E:/vGa_genes_AML.csv", header= 1)
# genes_vGa.drop(genes_vGa.columns[1:4], axis=1, inplace=True)
# genes_vGa.drop(genes_vGa.columns[4:7], axis=1, inplace=True)
# df_vGa = genes_vGa.sort_values('Gene')
# genes_AML = df_vGa['Gene'].to_list()
    

#Ganan-Gómez (MDS)
MDSpath = r'E:/AML_project/DATASETS/Ganan/pacientes/genecounts'
healthypath = r'E:/AML_project/DATASETS/Ganan/sanos/genecounts'
df_hu = pd.read_csv(MDSpath +'/GSM4079127_hu-CD34.txt.gz',sep='\t') #read file
df_47 = pd.read_csv(healthypath +'/GSM4079129_47-CD34.txt.gz',sep='\t') #read file
df_47 = df_47.reset_index() #reset index
df_hu = df_hu.reset_index() #reset index
df_47.rename(columns={'index': 'Gene'}, inplace=True) #name column with genes (previous index)
df_hu.rename(columns={'index': 'Gene'}, inplace=True) #name column with genes (previous index)
#Formating genes in GG data
for n in range(0, df_47.count()[0]): #for each data in the data frame
    df_47.at[n,'Gene'] = df_47.at[n,'Gene'][16:] #in each gene name eliminate the first 16 characters
for n in range(0, df_hu.count()[0]): #for each data in the data frame
    df_hu.at[n,'Gene'] = df_hu.at[n,'Gene'][16:]#in each gene name eliminate the first 16 characters
df_47 = df_47.sort_values('Gene') #sort values alphabetically
df_hu = df_hu.sort_values('Gene') #sort values alphabetically
genes_hu = df_hu['Gene'].to_list() #generate list of the genes
genes_47 = df_47['Gene'].to_list() #generate list of the genes
genes_MDS = list(set(genes_hu) & set(genes_47))

#Simona Colla (DC)
DCpath = r'E:/AML_project/DATASETS/Colla/genelist_DC.csv'
df_DC = pd.read_csv(DCpath) #read file
genes_DC = df_DC['Gene'].to_list() #generate list of the genes

Generate list of intersecting genes between selected samples

In [4]:
def gene_intersection(first_list, second_list):
    global gene_list
    gene_list = list(set(first_list) & set(second_list))
    global shared_genes
    shared_genes = np.array(gene_list)

In [5]:
    #(3 datasets)
#Rodríguez
# gene_list = list(set(genes_MDS) & set(genes_AML) & set(genes_FA))
# shared_genes = np.array(gene_list)
#Colla
# gene_list = list(set(genes_MDS) & set(genes_AML) & set(genes_DC))
# shared_genes = np.array(gene_list)

    #(2 datasets)
#van Galen and Rodríguez
# gene_intersection(genes_AML, genes_FA)
    #Ganan-Gómez and Rodríguez
# gene_intersection(genes_MDS, genes_FA)
    #van Galen and Colla
# gene_intersection(genes_AML, genes_DC)
    #Ganan-Gómez and Colla
# gene_intersection(genes_MDS, genes_DC)

    #Colla and Rodríguez
gene_intersection(genes_DC, genes_FA)

Data parsing

In [31]:
##Directories
    
    #Rodríguez
dir_Rod_BM = "E:/AML_project/DATASETS/Rodriguez/sanos/genecounts/"
dir_Rod_FA = "E:/AML_project/DATASETS/Rodriguez/pacientes/genecounts/"

    #van Galen
dir_vGa_BM = "E:/AML_project/DATASETS/vanGalen/sanos/genecounts/analyze/"
dir_vGa_AML = "E:/AML_project/DATASETS/vanGalen/pacientes/genecounts/D0/"
dir_vGa_BM_anno = "E:/AML_project/DATASETS/vanGalen/sanos/annotations/analyze/"
dir_vGa_AML_anno = "E:/AML_project/DATASETS/vanGalen/pacientes/annotations/D0/"

    #Ganan-Gómez
dir_GG_BM = "E:/AML_project/DATASETS/Ganan/sanos/genecounts/"
dir_GG_MDS = "E:/AML_project/DATASETS/Ganan/pacientes/genecounts/"
dir_GG_BM_anno = "E:/AML_project/DATASETS/Ganan/sanos/annotations/"
dir_GG_MDS_anno = "E:/AML_project/DATASETS/Ganan/pacientes/annotations/"

    #Colla
dir_Co_BM = "E:/AML_project/DATASETS/Colla/sanos/genecounts/"
dir_Co_BM_anno = "E:/AML_project/DATASETS/Colla/sanos/annotations/"

    #Output
dir_out_data = "E:/AML_project/DATASETS/building/" #data
dir_out_meta = "E:/AML_project/METADATA/" #metadata

    -Rodríguez

In [32]:
##Rodríguez
    
    #genecounts
    
#Healthy individuals (BM)
file_dir_rod = []
file_names_rod = []
sample_names_rod = []
anno_names_rod = []

for i in os.listdir(dir_Rod_BM):
    path = os.path.join(dir_Rod_BM,i)
    name = i.replace(".csv","")
    name_anno = i.replace(".csv","_anno")
    file_dir_rod.append(path)
    file_names_rod.append(i)
    sample_names_rod.append(name)
    anno_names_rod.append(name_anno)

    -vanGalen

In [33]:
##van Galen

    #genecounts
        
file_dir_vGa = []
file_names_vGa = []
sample_names_vGa = []

#Healthy individuals (BM)
for i in os.listdir(dir_vGa_BM):
    path = os.path.join(dir_vGa_BM,i)
    name = i.replace(".dem.txt.gz","")
    name = name[11:]
    file_dir_vGa.append(path)
    file_names_vGa.append(i)
    sample_names_vGa.append(name)
    
#Patients (AML)
for i in os.listdir(dir_vGa_AML):
    path = os.path.join(dir_vGa_AML,i)
    name = i.replace(".dem.txt.gz","")
    name = name[11:]
    file_dir_vGa.append(path)
    file_names_vGa.append(i)
    sample_names_vGa.append(name)
    
    #annotations

file_dir_vGa_anno = []
file_names_vGa_anno = []
anno_names_vGa = []

#Healthy individuals (BM)
for i in os.listdir(dir_vGa_BM_anno):
    path = os.path.join(dir_vGa_BM_anno,i)
    name = i.replace(".anno.txt.gz","_anno")
    name = name[11:]
    file_dir_vGa_anno.append(path)
    file_names_vGa_anno.append(i)
    anno_names_vGa.append(name)
    
#Patients (AML)
for i in os.listdir(dir_vGa_AML_anno):
    path = os.path.join(dir_vGa_AML_anno,i)
    name = i.replace(".anno.txt.gz","_anno")
    name = name[11:]
    file_dir_vGa_anno.append(path)
    file_names_vGa_anno.append(i)
    anno_names_vGa.append(name)

    -Ganan-Gómez

In [34]:
##Ganan-Gómez

    #genecounts
        
file_dir_GG = []
file_names_GG = []
sample_names_GG = []
anno_names_GG = []


#Healthy individuals (BM)
for i in os.listdir(dir_GG_BM):
    path = os.path.join(dir_GG_BM,i)
    name = i.replace(".txt.gz","")
    name = name[11:]
    name_anno = i.replace(".txt.gz","_anno")
    name_anno = name_anno[11:]
    file_dir_GG.append(path)
    file_names_GG.append(i)
    sample_names_GG.append(name)
    anno_names_GG.append(name_anno)
    
#Patients (AML)
for i in os.listdir(dir_GG_MDS):
    path = os.path.join(dir_GG_MDS,i)
    name = i.replace(".txt.gz","")
    name = name[11:]
    name_anno = i.replace(".txt.gz","_anno")
    name_anno = name_anno[11:]
    file_dir_GG.append(path)
    file_names_GG.append(i)
    sample_names_GG.append(name)
    anno_names_GG.append(name_anno)
    
    
#Out directories

dir_anno_GG_0 = dir_GG_BM_anno + anno_names_GG[0] 
dir_anno_GG_1 = dir_GG_BM_anno + anno_names_GG[1]
dir_anno_GG_2 = dir_GG_MDS_anno + anno_names_GG[2]
dir_anno_GG_3 = dir_GG_MDS_anno + anno_names_GG[3]

    -Colla

In [35]:
##Colla

    #genecounts
        
file_dir_Co = []
file_names_Co = []
sample_names_Co = []
anno_names_Co = []


#Healthy individuals (BM)
for i in os.listdir(dir_Co_BM):
    path = os.path.join(dir_Co_BM,i)
    name = i.replace(".csv.gz","")
    name = name[11:]
    name_anno = i.replace(".csv.gz","_anno")
    name_anno = name_anno[11:]
    file_dir_Co.append(path)
    file_names_Co.append(i)
    sample_names_Co.append(name)
    anno_names_Co.append(name_anno)
    
    
#Out directories fro annotations

dir_anno_Co_0 = dir_Co_BM_anno + anno_names_Co[0] 
dir_anno_Co_1 = dir_Co_BM_anno + anno_names_Co[1]
dir_anno_Co_2 = dir_Co_BM_anno + anno_names_Co[2]
dir_anno_Co_3 = dir_Co_BM_anno + anno_names_Co[3]

In [36]:
#Combination of datasets

# comb = "RvG"
# comb = "RGG"
# comb = "CovG"
# comb = "CoGG"
# comb = "CoR"
comb = "R"

Generate general dataframes for genecounts and annotations to merge datasets

In [37]:
#Generate empty dataframe as template to merge others
data = []
all_samples = pd.DataFrame(data,index = gene_list)
anno = []
all_anno = pd.DataFrame(anno, columns = ["PredictionRefined","CellType"])

Sampling and annotating composite dataset to train model

    Just run the datasets that you want to merge

        -vanGalen

In [43]:
##vanGalen (AML)

    #Loading data

#Load cell count file
for n in range(0, len(file_dir_vGa)):
    sample_names_vGa[n] = pd.read_csv(file_dir_vGa[n],sep='\t') #Read file   
    
#Load annotation file 
for n in range(0, len(file_dir_vGa_anno)):
    anno_names_vGa[n] = pd.read_csv(file_dir_vGa_anno[n],sep='\t') #Read file
    
    #Sampling intersecting gene

#Select only genes found in the intersection and create gene(row) and cell(column) dataframe 
for n in range(0, len(file_dir_vGa)):
    gene_num_vGa = [i for i, x in enumerate(sample_names_vGa[n]['Gene']) if x in gene_list] #selecting row numbers of the genes found in the intersection
    sample_names_vGa[n] = sample_names_vGa[n].iloc[gene_num_vGa, :] #create dataframe with only the row numbers of the genes found in the intersection
    sample_names_vGa[n].index = gene_list #change index to list of gene names

    #Sampling malignant and non-malignant cells
        
#Cells that where classified by differentiation (-like = malignant in most cases)
lineg_like_cells = ['HSC-like', 'Prog-like', 'GMP-like', 'ProMono-like'] 
lineg_cells = ['HSC', 'Prog', 'GMP', 'ProMono']

#Define and identify non-malignant and malignant cells (only the lineages that wished to be used) in the annotation file
malignant_cells = ['malignant']
nonmalignant_cells = ['normal', "unclear"]

#Extract the malignant and non-malignant cells from each file (cell counts and annotations)
for n in range(0, len(file_dir_vGa_anno)):
    
#identifying malignant and non-malignant cells that are lineage negative from the annotation files
    num_lineg_like = [i for i,x in enumerate(anno_names_vGa[n].CellType) if x in lineg_like_cells]  #selecting row numbers of lineage neg-like cells
    num_lineg = [i for i,x in enumerate(anno_names_vGa[n].CellType) if x in lineg_cells] #selecting row numbers of lineage-neg cells
    
    num_malignant = [i for i,x in enumerate(anno_names_vGa[n].PredictionRefined) if x in malignant_cells]  #selecting row numbers of the malignant cells in annotation file
    num_nonmalignant = [i for i,x in enumerate(anno_names_vGa[n].PredictionRefined) if x in nonmalignant_cells] #selecting row numbers of the non-malignant cells in annotation file
    
    num_mal_lineg = [x for x in num_malignant if x in num_lineg_like] #listing the row numbers that are lineage negative and malignant
    num_nonmal_lineg  = [x for x in num_nonmalignant if x in num_lineg] #listing the row numbers that are lineage negative and non-malignant
    
    barcodes_vGa = list(anno_names_vGa[n].Cell[num_nonmal_lineg]) + list(anno_names_vGa[n].Cell[num_mal_lineg]) #creating list with cell IDs of malignant and non-malignant cells to identify them in cell count file
    general_col = [i for i,x in enumerate(sample_names_vGa[n].columns) if x in barcodes_vGa]  #selecting row numbers of the malignant and non-malignant cells in the cell count file
    
#Extracting selected cells (only malignant and non-maliignant)
    sample_names_vGa[n] = sample_names_vGa[n].iloc[:,general_col] #Cell counts
    anno_num_vGa = [i for i, x in enumerate(anno_names_vGa[n]['Cell']) if x in barcodes_vGa]
    anno_names_vGa[n] = anno_names_vGa[n].iloc[anno_num_vGa]

    #Merge dataframes into the general one

#Merging all files
for n in range(1, len(file_dir_vGa_anno)):
    all_samples = pd.merge(all_samples, sample_names_vGa[n], left_index=True, right_index=True, suffixes=('', '_drop')) #append all dataframes into a single one
    all_anno = pd.concat([all_anno, anno_names_vGa[n]], axis=0,  ignore_index=True) #append all dataframes into a single one
all_samples.drop([col for col in all_samples.columns if 'drop' in col], axis=1, inplace=True) #there is a duplication of columns

#ONLY IF IT IS THE LAST DATASET ADDED
    #Reseting index to gene numbers from vGa to create anndata object
# all_samples.index = gene_num_vGa
# del all_samples['Gene']

        -Rodríguez

In [38]:
##Rodriguez (FA)

    #Loading data

#Load cell count file (scRNAseq)
for n in range(0, len(file_dir_rod)):
    sample_names_rod[n] = pd.read_csv(file_dir_rod[n], sep='\t') #Read file
    sample_names_rod[n] = sample_names_rod[n].rename({'Unnamed: 0': 'Gene'}, axis=1) #Rename gene column
    
    #Sampling intersecting genes
        
#Select only genes found in the intersection and create gene(row) and cell(column) dataframe
    gene_num_rod = [i for i, x in enumerate(sample_names_rod[n]['Gene']) if x in gene_list] #selecting row numbers of the genes found in the intersection
    df_inter = sample_names_rod[n].iloc[gene_num_rod] #create dataframe with only the row numbers of the genes found in the intersection
    sample_names_rod[n] = df_inter.sort_values('Gene') #sort gene rows alphabetically to match vanGalen format
    sample_names_rod[n].index = gene_list #changing index to gene names (alphabetically ordered) to match vGa data frame indexes
    sample_names_rod[n]= sample_names_rod[n].drop('Gene', axis=1) #remove gene column
    sample_names_rod[n] = sample_names_rod[n].add_suffix("_FA")
    
    #Annotating data based on vanGalen classification

#Creata annotation file with celltype and class 
    columns = sample_names_rod[n].columns.values #get list of cell IDs (column names)
    columns = columns[0:,] #extract cell IDs only (columns, without the first one = gene)
    anno_names_rod[n] = pd.DataFrame(columns) #create dataframe with cell IDs as rows
    anno_names_rod[n] = anno_names_rod[n].rename({0: 'Cell'}, axis=1) #name the first column as Cell
    anno_names_rod[n] = pd.DataFrame(anno_names_rod[n],columns=['Cell','PredictionRefined']) #add columns based on vanGalen annotations (celltype and class)
    anno_names_rod[n]['PredictionRefined'] = "normal" #define all cells as normal (data is from a healthy individual)
    anno_names_rod[n]['CellType'] = 'lin-' #define all cells as lin- (data is from lin- cells)
    
    #Show summary of the amount of cells per celltype and class
    # print("The sample has:")
    # print(Counter(annohealthy1.PredictionRefined)) #print number of cells in class
    # print(Counter(annohealthy1.CellType)) #print number of cells in each celltype
    
    #Merge dataframes into the general one
    
#Merging all files
    all_samples = pd.merge(all_samples, sample_names_rod[n], left_index=True, right_index=True, suffixes=('', '_drop')) #append all dataframes into the van Galen merge
    all_anno = pd.concat([all_anno, anno_names_rod[n]], axis=0,  ignore_index=True) #append all dataframes into a single one
all_samples.drop([col for col in all_samples.columns if 'drop' in col], axis=1, inplace=True) #there is a duplication of columns

#ONLY IF IT IS THE LAST DATASET ADDED
    #Reseting index to gene numbers from vGa to create anndata object
# all_samples.index = gene_num_rod
# del all_samples['Gene']

        -Colla

In [141]:
##Colla

    #Loading data
donors = ["1","2","3","4"]

#Load cell count file
for n in range(0, len(file_dir_Co)):
    sample_names_Co[n] = pd.read_csv(file_dir_Co[n]) #Read file
    sample_names_Co[n]['Gene'] = genes_DC #change gene IDs to gene names (previously obtained)
    sample_names_Co[n] = sample_names_Co[n].drop_duplicates(['Gene']) #Eliminating gene duplicates (only Ganan-Gómez data has them)
    del sample_names_Co[n]['Unnamed: 0'] #remove column with gene IDs 


    #Sampling intersecting genes
        
#Select only genes found in the intersection and create gene(row) and cell(column) dataframe
    gene_num_Co = [i for i, x in enumerate(sample_names_Co[n]['Gene']) if x in gene_list] #selecting row numbers of the genes found in the intersection
    df_inter = sample_names_Co[n].iloc[gene_num_Co] #create dataframe with only the row numbers of the genes found in the intersection
    sample_names_Co[n] = df_inter.sort_values('Gene') #sort gene rows alphabetically to match vanGalen format
    sample_names_Co[n].index = gene_list #changing index to gene names (alphabetically ordered) to match vGa data frame indexes
    sample_names_Co[n]= sample_names_Co[n].drop('Gene', axis=1) #remove gene column
    sample_names_Co[n] = sample_names_Co[n].add_suffix("_Healthy_" + donors[n] + "_DC")
    
    #Annotating data based on vanGalen classification

#Creata annotation file with celltype and class 
    columns = sample_names_Co[n].columns.values #get list of cell IDs (column names)
    columns = columns[0:,] #extract cell IDs only (columns, without the first one = gene)
    anno_names_Co[n] = pd.DataFrame(columns) #create dataframe with cell IDs as rows
    anno_names_Co[n] = anno_names_Co[n].rename({0: 'Cell'}, axis=1) #name the first column as Cell
    anno_names_Co[n] = pd.DataFrame(anno_names_Co[n],columns=['Cell','PredictionRefined']) #add columns based on vanGalen annotations (celltype and class)
    anno_names_Co[n]['CellType'] = 'lin-' #define all cells as lin- (data is from lin- cells)
    anno_names_Co[n]['PredictionRefined'] = "normal" #define all cells as normal (data is from a healthy individual)
    
    if n == 0:
        anno_names_Co[n].to_csv(dir_anno_Co_0 + ".csv", index = False)
    if n == 1:
        anno_names_Co[n].to_csv(dir_anno_Co_1 + ".csv", index = False)
    if n == 2:
        anno_names_Co[n].to_csv(dir_anno_Co_2 + ".csv", index = False)
    if n == 3:
        anno_names_Co[n].to_csv(dir_anno_Co_3 + ".csv", index = False)
    
    #Show summary of the amount of cells per celltype and class
    # print("The sample has:")
    # print(Counter(annohealthy1.PredictionRefined)) #print number of cells in class
    # print(Counter(annohealthy1.CellType)) #print number of cells in each celltype
    
    #Merge dataframes into the general one
    
#Merging all files
    all_samples = pd.merge(all_samples, sample_names_Co[n], left_index=True, right_index=True, suffixes=('', '_drop')) #append all dataframes into the van Galen merge
    all_anno = pd.concat([all_anno, anno_names_Co[n]], axis=0,  ignore_index=True) #append all dataframes into a single one
all_samples.drop([col for col in all_samples.columns if 'drop' in col], axis=1, inplace=True) #there is a duplication of columns

#ONLY IF IT IS THE LAST DATASET ADDED
    #Reseting index to gene numbers from vGa to create anndata object
# all_samples.index = gene_num_Co


        -Ganan-Gómez

In [60]:
##Ganan-Gómez

    #Loading data

#Load cell count file
for n in range(0, len(file_dir_GG)):
    sample_names_GG[n] = pd.read_csv(file_dir_GG[n],sep='\t') #Read file
    sample_names_GG[n] = sample_names_GG[n].reset_index() #reset index
    sample_names_GG[n].rename(columns={'index': 'Gene'}, inplace=True) #name column with genes (previous index) 
#Formating genes in GG data (eliminate gene ID number)
    for i in range(0, sample_names_GG[n].count()[0]):
        sample_names_GG[n].at[i,'Gene'] = sample_names_GG[n].at[i,'Gene'][16:]

#Eliminating gene duplicates (only Ganan-Gómez data has them)
    sample_names_GG[n] = sample_names_GG[n].drop_duplicates(['Gene'])

    #Sampling intersecting genes
        
#Select only genes found in the intersection and create gene(row) and cell(column) dataframe
    gene_num_GG = [i for i, x in enumerate(sample_names_GG[n]['Gene']) if x in gene_list] #selecting row numbers of the genes found in the intersection
    df_inter = sample_names_GG[n].iloc[gene_num_GG] #create dataframe with only the row numbers of the genes found in the intersection
    sample_names_GG[n] = df_inter.sort_values('Gene') #sort gene rows alphabetically to match vanGalen format
    sample_names_GG[n].index = gene_list #changing index to gene names (alphabetically ordered) to match vGa data frame indexes
    sample_names_GG[n]= sample_names_GG[n].drop('Gene', axis=1) #remove gene column
    
    #Annotating data based on vanGalen classification

#Creata annotation file with celltype and class 
    columns = sample_names_GG[n].columns.values #get list of cell IDs (column names)
    columns = columns[0:,] #extract cell IDs only (columns, without the first one = gene)
    anno_names_GG[n] = pd.DataFrame(columns) #create dataframe with cell IDs as rows
    anno_names_GG[n] = anno_names_GG[n].rename({0: 'Cell'}, axis=1) #name the first column as Cell
    anno_names_GG[n] = pd.DataFrame(anno_names_GG[n],columns=['Cell','PredictionRefined','Patient_type']) #add columns based on vanGalen annotations (celltype and class)
    anno_names_GG[n]['CellType'] = 'lin-' #define all cells as lin- (data is from lin- cells)
    if n < 2:
        anno_names_GG[n]['PredictionRefined'] = "normal" #define all cells as normal (data is from a healthy individual)
    else:
        anno_names_GG[n]['PredictionRefined'] = "malignant" #define all cells as normal (data is from a healthy individual)
        
    if n == 0:
        anno_names_GG[n]['Patient_type'] = "normal" #define patient as normal
        anno_names_GG[n].to_csv(dir_anno_GG_0 + ".csv", index = False)
    if n == 1:
        anno_names_GG[n]['Patient_type'] = "normal" #define patient as normal
        anno_names_GG[n].to_csv(dir_anno_GG_1 + ".csv",index = False)
    if n == 2:
        anno_names_GG[n]['Patient_type'] = "CMP" #define patient as CMP
        anno_names_GG[n].to_csv(dir_anno_GG_2 + ".csv",index = False)
    if n == 3:
        anno_names_GG[n]['Patient_type'] = "GMP" #define patient as GMP
        anno_names_GG[n].to_csv(dir_anno_GG_3 + ".csv",index = False)
        
    del anno_names_GG[n]['Patient_type'] #this column will be used ofr other analysis but not for the model building
            
    #Show summary of the amount of cells per celltype and class
    # print("The sample has:")
    # print(Counter(annohealthy1.PredictionRefined)) #print number of cells in class
    # print(Counter(annohealthy1.CellType)) #print number of cells in each celltype
    
    #Merge dataframes into the general one
    
#Merging all files
    all_samples = pd.merge(all_samples, sample_names_GG[n], left_index=True, right_index=True, suffixes=('', '_drop')) #append all dataframes into the van Galen merge
    all_anno = pd.concat([all_anno, anno_names_GG[n]], axis=0,  ignore_index=True) #append all dataframes into a single one
all_samples.drop([col for col in all_samples.columns if 'drop' in col], axis=1, inplace=True) #there is a duplication of columns

#ONLY IF IT IS THE LAST DATASET ADDED
    #Reseting index to gene numbers from vGa to create anndata object
# all_samples.index = gene_num_GG
# del all_samples['Gene']

In [39]:
#Run at the end of every combination to get the IDs of the cells of the dataframe
barcodes_general = all_samples.columns.values.tolist()

Check the sum of the cells

In [ ]:
#Check the concatenations was done correctly

#vanGalen
# sumvGa = 0
# for n in range(0, len(file_dir_vGa)):
#     b = sample_names_vGa[n].shape[1]
#     sumvGa = sumvGa + b
# print("The total number of cells in van Galen is:", sumvGa)

#Rodríguez
sumRod = 0
for n in range(0, len(file_dir_rod)):
    b = sample_names_rod[n].shape[1]
    sumRod = sumRod + b
print("The total number of cells in Rodríguez is:", sumRod)

# #Ganan-Gómez
sumGG = 0
for n in range(0, len(file_dir_GG)):
    b = sample_names_GG[n].shape[1]
    sumGG = sumGG + b
print("The total number of cells in Ganan-Gómez is:", sumGG)

#Colla
# sumCo = 0
# for n in range(0, len(file_dir_Co)):
#     b = sample_names_Co[n].shape[1]
#     sumCo = sumCo + b
# print("The total number of cells in Colla is:", sumCo)

Separating malignant and non-malignant cells in diferent files

In [ ]:
#****Separating malignant and non-malignant cells in diferent files** (this data can be used for further analysis)

    #Identifying malignant and non-malignant cell #s
    barcodes_malignant = list(anno_names_vGa[n].Cell[num_mal_lineg]) #creating list with cell IDs of malignant cells to identify them in cell count file
    barcodes_nonmalignant = list(anno_names_vGa[n].Cell[num_nonmal_lineg]) #creating list with cell IDs of non-malignant cells to identify them in cell count file

    #Identify non-malignant and malignant cells from the annotation file (cell #) in the cell count file
    malignant_col = [i for i,x in enumerate(sample_names_vGa[n].columns) if x in barcodes_malignant]  #selecting row numbers of the malignant cells in cell count file
    nonmalignant_col = [i for i,x in enumerate(sample_names_vGa[n].columns) if x in barcodes_nonmalignant] #selecting row numbers of the non-malignant cells in cell count file

    #Generate malignant and non-malingant dataframes (this can be used later)
    malignant = sample_names_vGa[n].iloc[:,malignant_col] #dataframe with cells that are malignant
    nonmalignant = sample_names_vGa[n].iloc[:,nonmalignant_col] #dataframe with cells that are non-malignant

    #Show summary of the amount of cells per celltype and class
    # print("The sample has:")
    # print(Counter(anno_names_vGa[n].CellType))
    # print(len(num_mal_lineg))
    # print(len(num_nonmal_lineg))

Saving output files

In [40]:
#Save data

    #Merged dataframe
all_samples.to_csv(dir_out_data + "all_samples_" + comb + ".csv")

    #Merged annotations
all_anno.to_csv(dir_out_data + "all_anno_" + comb + ".csv")

#Save Metadata

    #List of intersecting genes
with open(dir_out_meta + "gene_list_" + comb + ".txt", "w") as output:
        output.write(str(gene_list))
        
    #List of cell IDs 
with open(dir_out_meta + "barcodes_general_" + comb + ".txt", "w") as output:
        output.write(str(barcodes_general))

In [63]:
%%notify
print("Finished!")

Finished!


<IPython.core.display.Javascript object>